In [10]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
df_jan23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
#Question 1
len(df_jan23.columns) #answer: 19

19

In [7]:
df_jan23['duration'] = df_jan23.tpep_dropoff_datetime - df_jan23.tpep_pickup_datetime
df_jan23.duration = df_jan23.duration.apply(lambda td: td.total_seconds() / 60)

df_feb23['duration'] = df_feb23.tpep_dropoff_datetime - df_feb23.tpep_pickup_datetime
df_feb23.duration = df_feb23.duration.apply(lambda td: td.total_seconds() / 60)


In [8]:
#Question 2
df_jan23['duration'].describe()['std'] #answer: 42.59

42.594351241920904

In [9]:
#Question 3
initial_size = len(df_jan23)
df_jan23 = df_jan23[(df_jan23.duration >= 1) & (df_jan23.duration <= 60)]
df_feb23 = df_feb23[(df_feb23.duration >= 1) & (df_feb23.duration <= 60)]
after_size = len(df_jan23)
print(f'Percentage left: {after_size/initial_size * 100:.2f}%') #answer: 98.12%


Percentage left: 98.12%


In [10]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan23[categorical] = df_jan23[categorical].astype(str)
df_feb23[categorical] = df_feb23[categorical].astype(str)

In [11]:
train_dicts = df_jan23[categorical + numerical].to_dict(orient='records')
val_dicts = df_feb23[categorical + numerical].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_jan23['duration'].values

In [13]:
#Question 4:
X_train.shape[1]

516

In [14]:
#Question 5
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred, squared=False)
print(f'RMSE: {mse:.2f}') #answer: 7.65

RMSE: 7.65


In [15]:
#Question 6:
X_val = dv.transform(val_dicts)
y_val = df_feb23['duration'].values
y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)
print(f'Validation RMSE: {mse:.2f}') #answer: 7.64


Validation RMSE: 7.81
